In [12]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import textwrap
folder = "/media/data/software/database/mibig_3.1"

In [13]:
def draw_molecule_grid(df, smiles_col, name_col, distance_col, n_cols=5, save_path=None, max_name_width=25):
    # Sort the dataframe by 'Distance' column
    df = df.sort_values(by=distance_col).reset_index(drop=True)
    
    # Number of rows and columns
    n_rows = len(df) // n_cols + int(len(df) % n_cols > 0)
    
    # Increase image size and DPI for better quality
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols*4, n_rows*5), dpi=300)
    
    # Use the custom font
    font_path = '/home/iwe66/.local/share/fonts/AvenirNextLTPro-Bold.otf'
    font_prop = FontProperties(fname=font_path)
    
    for i, (index, row) in enumerate(df.iterrows()):
        # Convert SMILES to a molecule object
        mol = Chem.MolFromSmiles(row[smiles_col])
        
        if mol is not None:
            # Draw the molecule with higher resolution
            img = Draw.MolToImage(mol, size=(300, 300), kekulize=True)
            ax = axes[i // n_cols, i % n_cols]
            ax.imshow(img)
            ax.axis('off')
            
            # Ensure the name is a string and handle NaN values
            name = str(row[name_col]) if pd.notna(row[name_col]) else "Unnamed Molecule"
            # Wrap the name if it's too long
            wrapped_name = textwrap.fill(name, width=max_name_width)
            
            # Add the subtitle with name and distance, using the custom font
            subtitle = f"{wrapped_name}\nDistance: {row[distance_col]:.2f}"
            ax.set_title(subtitle, fontsize=12, fontproperties=font_prop)
    
    # Hide any remaining empty subplots
    for j in range(i + 1, n_rows * n_cols):
        fig.delaxes(axes.flatten()[j])
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=600, bbox_inches='tight')
    else:
        plt.show()

In [16]:
df = pd.read_csv('out/glob_G2d_0.85.csv')

In [17]:
# Example usage:
# Assuming df is your dataframe
draw_molecule_grid(df, smiles_col='smiles', name_col='mibig_accession', distance_col='distance')